# 상의/하의 추천 시스템
---

## 문제 인식
### 기존 추천시스템의 문제
유사도 기반 추천 알고리즘이므로, 기존 제품과 유사한 것만 추천해준다.  
(예시 : 상의 이미지가 들어올 경우, 비슷한 상의만 추천해준다)

### 해결하고자 하는 문제
제품이 들어올 경우 이와 어울리는 조합을 추천해주고 싶다  
(예시 : 상의가 들어오면 거기에 어울리는 하의/아우터/신발 등을 추천해주고 싶다)

## 문제 정의
### 추천해주고 싶은 옷의 조합의 범위는?
일단은 크게 3가지로만 시도한다 (상의/하의/아우터)  
남자 : 상의/하의(바지)/아우터
여자 : 상의/하의(바지,치마)/아우터
### 어울림이란 무엇인가
1. 종류  
트레이닝 상의-트레이닝 하의  
터틀넥 - 면바지  
-> 소분류 정보 필요
2. 색깔  
사전에 정의된 색조합이 있음 -> 색상 정보 필요
3. 계절
겨울옷이랑 여름옷이 매칭되면 이상함 -> 계절 정보 필요

### 사용자 취향 vs 객관적 어울림
사용자의 취향을 무조건 앞서서 보자

## 문제해결 방식
### 어울림 척도 정의
1. 상의/하의 종류간의 어울림 정도를 Attribute Matrix로 저장한다
2. 상의를 기반으로 상의/아우터간의 어울림 정도를 matrix로 저장한다.
3. ~~색상간의 어울림 정도를 matrix로 저장한다.~~-> 색상은 그냥 랜덤으로 하자
4. ~~계절정보를 반영한다~~-> 계절도 일단은 없는걸로 하자

or 

~크라우드 소싱(상의/하의/아우터 조합)~

### Attribution Matrix
상의x,하의 y간의 어울림정도를 0-1사이 값으로 표현  
Attribution Matrix 값이 높을수록 추천 우선순위  

Attribution Matrix의 개인화/수정은?  
->개인화는 쉽지 않음. DB를 새로 만들어야 함  
-> 수정도 쉽지 않음. 유저의 피드백을 받아서 정량적 계산을 통해 넣어야 함

In [1]:
import os

os.chdir('..')

In [19]:
import pandas as pd
import json
import pymysql

In [3]:
# 남성/여성 카테고리 받아오기
# 이것도 SQL통해 자동화할수있으면 해야함

male_up= """반팔 티셔츠
긴팔 티셔츠
민소매 티셔츠
셔츠/블라우스
피케/카라 티셔츠
맨투맨/스웨트셔츠
후드 스웨트셔츠/후드 집업
니트/스웨터/카디건
베스트
기타 상의"""

male_low = """데님 팬츠 
코튼 팬츠 
수트 팬츠/슬랙스 
트레이닝/조거 팬츠
숏 팬츠 
레깅스 
기타 바지"""

male_out = """블루종/MA-1
레더/라이더스 재킷
트러커 재킷
수트/블레이저 재킷
아노락 재킷
나일론/코치 재킷
트레이닝 재킷
스타디움 재킷
환절기 코트 
겨울 싱글 코트
겨울 기타 코트
롱 패딩/롱 헤비 아우터
숏 패딩/숏 헤비 아우터
패딩 베스트 
사파리/헌팅 재킷
기타 아우터"""

female_up= """반팔 티셔츠
긴팔 티셔츠
민소매 티셔츠
셔츠/블라우스
피케/카라 티셔츠
맨투맨/스웨트셔츠
후드 스웨트셔츠/후드 집업
니트/스웨터/카디건
베스트
기타 상의"""

female_low = """데님 팬츠 
코튼 팬츠 
수트 팬츠/슬랙스 
트레이닝/조거 팬츠
숏 팬츠 
레깅스 
기타 바지
미니 스커트
미디 스커트
롱 스커트"""


female_out = """블루종/MA-1
레더/라이더스 재킷
트러커 재킷
수트/블레이저 재킷
아노락 재킷
나일론/코치 재킷
트레이닝 재킷
스타디움 재킷
환절기 코트 
겨울 싱글 코트
겨울 기타 코트
롱 패딩/롱 헤비 아우터
숏 패딩/숏 헤비 아우터
패딩 베스트 
사파리/헌팅 재킷
기타 아우터 """

female_one="""미니 원피스
미디 원피스
맥시 원피스
점프수트"""

In [4]:
male_up = male_up.split('\n')
male_low = male_low.split('\n')
male_out = male_out.split('\n')
female_up = female_up.split('\n')
female_low = female_low.split('\n')
female_out = female_out.split('\n')
female_one = female_one.split('\n')

In [5]:
# Attribution Matrix 초기화(처음에는 하드코딩)
male_uplow = pd.read_csv('prep/male_uplow.csv',sep='\t',header=None)
male_upout = pd.read_csv('prep/male_upout.csv',sep='\t',header=None)
female_uplow = pd.read_csv('prep/female_uplow.csv',sep='\t',header=None)
female_upout = pd.read_csv('prep/female_upout.csv',sep='\t',header=None)
female_oneout = pd.read_csv('prep/female_oneout.csv',sep='\t',header=None)

In [6]:
#인덱스 설정
male_uplow.columns = male_up
male_uplow.index = male_low

male_upout.columns = male_up
male_upout.index = male_out

female_uplow.columns = female_up
female_uplow.index = female_low

female_upout.columns = female_up
female_upout.index = female_out

female_oneout.columns = female_one
female_oneout.index = female_out

In [9]:
#Transpose DataFrame 획득
male_lowup = male_uplow.T
male_outup = male_upout.T
female_lowup = female_uplow.T
female_outup = female_upout.T
female_outone = female_oneout.T

In [17]:
def connect_db(dbinfo_path) :
    with open(dbinfo_path) as jsonfile :
        dbinfo = json.load(jsonfile)

    connection = pymysql.connect(host=dbinfo['host'],
                         port=dbinfo['port'],
                         user=dbinfo['user'], 
                         passwd=dbinfo['passwd'],
                         db=dbinfo['db'],
                         charset=dbinfo['charset'])
    
    return connection

In [44]:
# DB Class index <-> 카테고리 한글명 관계
# 아동은 제외하고 남/여 상의만 불러오기
# (아동의 경우는 태그 분류 학습용으로는 가능하지만, 추천용으로는 제외해야 함)
# 스포츠 -> 기능성 상의도 제외
db = connect_db('dbinfo-temp.json')
cur = db.cursor()

sql = 'SELECT id,major,middle,minor FROM CLOTHES_CLASS WHERE middle!="" AND major!="아동" AND middle!="스포츠/용품" AND middle!="가방";' # \으로 개행해도 SQL 정상 동작 확인
cur.execute(sql)

78

In [45]:
clothes_class ={}
for id,major,middle,minor in cur :
    clothes_class[id] = [major,middle,minor]

clothes_class = pd.DataFrame(clothes_class)

In [46]:
clothes_class

,52,53,54,55,56,57,58,59,60,61,...,120,121,122,123,124,125,126,127,128,141
0,남,여,남,여,여,남,남,여,남,여,...,여,남,남,여,여,남,여,남,여,남
1,상의,상의,상의,상의,상의,상의,상의,상의,상의,상의,...,바지,바지,바지,바지,스커트,스커트,스커트,스커트,스커트,원피스
2,반팔 티셔츠,반팔 티셔츠,긴팔 티셔츠,긴팔 티셔츠,민소매 티셔츠,민소매 티셔츠,셔츠/블라우스,셔츠/블라우스,피케/카라 티셔츠,피케/카라 티셔츠,...,레깅스,레깅스,기타 바지,기타 바지,미니 스커트,미니 스커트,미디 스커트,미디 스커트,롱 스커트,미디 원피스


## 아이템간 추천
---
추천의 시작 : 가장 중요한 것은 위시리스트 & 취향   
위시리스트/취향으로부터 우선 추천 -> 상의/하의/아우터에 대해 유사도 기반 각 3개씩 추천  
위시리스트와 어울리는 세트 추천 -> 상의로부터 하의 추천, 하의로부터 상의 추천, 상의로부터 아우터 추천하는 식으로 새로 만들어서 3개 추천  
사용자에게는 그냥 퉁쳐서 '취향과 어울리는 세트 상품입니다' 로 추천


### 카테고리 추천

In [ ]:
# 카테고리 선택 : 현재 상/하의로부터 가장 높은 어울림점수를 갖는 상/하의 카테고리 선택
# 아이템 선택 : 카테고리가 선택되면, 이로부터 DB 탐색. 마지막 등록 N개까지. 이중에 랜덤 하나 추천

#input 들어오는 형태
input_class_id = 52

In [47]:
input_major = clothes_class[input_class_id][0]
input_middle = clothes_class[input_class_id][1]
input_minor = clothes_class[input_class_id][2]

In [99]:
# 상의/하의 추천함수
def over_mean(scores:pd.Series) :
    return scores[scores>scores.mean()].index.values

def recommand_set(major,middle,minor) :
    male_map = {'상의':(male_uplow,male_upout),'바지':male_lowup,'아우터':male_outup}
    female_map = {'상의':(female_uplow,female_upout),'바지':female_lowup,'스커트':female_lowup,
                  '아우터':(female_outup,female_outone),'원피스':female_oneout}
    mf_map = {'남':male_map,'여':female_map}
    
    for df in mf_map[major][middle] :    
        print("현재 옷에 잘 어울리는 카테고리는",over_mean(df[minor]))

In [100]:
recommand_set('남','상의','반팔 티셔츠')

현재 옷에 잘 어울리는 카테고리는 ['데님 팬츠 ' '코튼 팬츠 ' '트레이닝/조거 팬츠' '숏 팬츠 ']
현재 옷에 잘 어울리는 카테고리는 ['블루종/MA-1' '나일론/코치 재킷' '트레이닝 재킷']


### 카테고리 내 상품 추천

In [104]:
# 원래 계획 : 카테고리가 결정되면, 거기서부터 색상 기반으로 맞춤형 추천을 한다
# 실제 구현 : 색상정보를 하기가 어려워서 그냥 랜덤하게 최신순으로 N개 뽑아서 추천해줌
# 혹은 여유가 된다면, feature extraction -> content based 추천으로 할 수있음
def recommand_item(major,minor) :
    db = connect_db('dbinfo-temp.json')
    cur = db.cursor()
    sql = "SELECT CLOTHES.id,CLOTHES.img FROM CLOTHES_AND_CLOTHES_CLASS \
            INNER JOIN CLOTHES ON  CLOTHES_AND_CLOTHES_CLASS.clothes_id = CLOTHES.id \
            INNER JOIN CLOTHES_CLASS ON CLOTHES_AND_CLOTHES_CLASS.clothes_class_id = CLOTHES_CLASS.id \
            WHERE CLOTHES_CLASS.major = '{}' AND CLOTHES_CLASS.minor ='{}'  \
            ORDER BY CLOTHES.updated DESC \
            LIMIT 5;".format(major,minor)
    cur.execute(sql)

    return cur

In [106]:
result = recommand_item('남','데님 팬츠')

In [107]:
for id,img in result :
    print(id,img)

48505 //image.msscdn.net/images/goods_img/20200306/1339640/1339640_1_500.jpg
48502 //image.msscdn.net/images/goods_img/20200204/1289815/1289815_2_500.jpg
48498 //image.msscdn.net/images/goods_img/20200204/1289405/1289405_1_500.jpg
48496 //image.msscdn.net/images/goods_img/20191216/1251930/1251930_2_500.jpg
48493 //image.msscdn.net/images/goods_img/20190201/947579/947579_12_500.jpg
